### Goal of notebook is to figure out the most frequent diseases in the MIMIC-III dataset and compare with top diseases in CSU dataset

In [25]:
import pandas as pd
import re
import numpy as np
import csv
import sys

from IPython.display import display, HTML


In [26]:
from icd9 import ICD9

In [23]:
mimic = pd.read_csv('/Users/ashley1/Documents/Stanford/Bustamante_lab/Vet_Data_Collaboration/Rivas_Zou_labs/CSU_Altud/R_directory/icd9NotesDataTable.csv')

display(mimic[:5])

,Unnamed: 0,HADM_ID,SUBJECT_ID,ICD9_CODE,CHARTDATE,DESCRIPTION,TEXT,Level2ICD,TopLevelICD,V9
0,1,167853,22532,01193-4254-42731-2639-2762-5070-5119-2113,2151-08-04,Report,Admission Date: [**2151-7-16**] Dischar...,NaN,11-425-427-263-276-507-511-211,cat:1-cat:8-cat:3-cat:9-cat:2
1,2,107527,13702,5191-49121-51881-486-2761-2449-311,2118-06-14,Report,Admission Date: [**2118-6-2**] Discharg...,NaN,519-491-518-486-276-244-311,cat:9-cat:3-cat:5
2,3,167118,13702,5191-5185-496-2762-45340-5533,2119-05-25,Report,Admission Date: [**2119-5-4**] D...,NaN,519-518-496-276-453-553,cat:9-cat:3-cat:8-cat:10
3,4,196489,13702,51884-5849-34830-49121-2760-4160-3594-5780-276...,2124-08-18,Report,Admission Date: [**2124-7-21**] ...,NaN,518-584-348-491-276-416-359-578-530-487-553-V0...,cat:9-cat:11-cat:6-cat:3-cat:8-cat:10-cat:19
4,5,135453,26880,80506-5070-42823-2930-4538-E882-4280-4011-V450...,2162-03-25,Report,Admission Date: [**2162-3-3**] D...,NaN,805-507-428-293-453-E882-401-V4502-427-V1254-8...,cat:18-cat:9-cat:8-cat:5-cat:19-cat:17


In [5]:
mimic[:50].to_csv('mimic_sample', sep='\t', index=False)

In [36]:
mimic.shape

(52722, 10)

### source for ICD-9 description file:
https://www.cms.gov/Medicare/Coding/ICD9ProviderDiagnosticCodes/codes.html

Note: apparently ICD9 hierarchies are rough at best and the recommendations are to map ICD9 codes over to SNOMED if looking to traverse hierarchies:
http://forums.ohdsi.org/t/lack-of-icd9-hierarchy/249/4

found github repo for traversing the ICD9 tree with descriptions:
https://github.com/sirrice/icd9

In [55]:
#may be easier to merge in a flat description file:
desc = pd.read_csv('/Users/ashley1/repos/icd9/icd9/CMS32_DESC_SHORT_DX.txt', sep='\t', names='A')
desc

,A
0,0010 Cholera d/t vib cholerae
1,0011 Cholera d/t vib el tor
2,0019 Cholera NOS
3,0020 Typhoid fever
4,0021 Paratyphoid fever a
5,0022 Paratyphoid fever b
6,0023 Paratyphoid fever c
7,0029 Paratyphoid fever NOS
8,0030 Salmonella enteritis
9,0031 Salmonella septicemia


In [60]:
desc['B'], desc['C'] = desc['A'].str.split(' ', 1).str
desc


,A,B,C
0,0010 Cholera d/t vib cholerae,0010,Cholera d/t vib cholerae
1,0011 Cholera d/t vib el tor,0011,Cholera d/t vib el tor
2,0019 Cholera NOS,0019,Cholera NOS
3,0020 Typhoid fever,0020,Typhoid fever
4,0021 Paratyphoid fever a,0021,Paratyphoid fever a
5,0022 Paratyphoid fever b,0022,Paratyphoid fever b
6,0023 Paratyphoid fever c,0023,Paratyphoid fever c
7,0029 Paratyphoid fever NOS,0029,Paratyphoid fever NOS
8,0030 Salmonella enteritis,0030,Salmonella enteritis
9,0031 Salmonella septicemia,0031,Salmonella septicemia


In [62]:
desc.drop(['A'], inplace=True, axis=1)
desc

,B,C
0,0010,Cholera d/t vib cholerae
1,0011,Cholera d/t vib el tor
2,0019,Cholera NOS
3,0020,Typhoid fever
4,0021,Paratyphoid fever a
5,0022,Paratyphoid fever b
6,0023,Paratyphoid fever c
7,0029,Paratyphoid fever NOS
8,0030,Salmonella enteritis
9,0031,Salmonella septicemia


In [70]:
desc.rename(columns={'conceptID':'conceptId', 'C':'term'}, inplace=True)
desc

,conceptId,term
0,0010,Cholera d/t vib cholerae
1,0011,Cholera d/t vib el tor
2,0019,Cholera NOS
3,0020,Typhoid fever
4,0021,Paratyphoid fever a
5,0022,Paratyphoid fever b
6,0023,Paratyphoid fever c
7,0029,Paratyphoid fever NOS
8,0030,Salmonella enteritis
9,0031,Salmonella septicemia


In [4]:
tree = ICD9('codes.JSON')


In [5]:
type(tree)

icd9.ICD9

In [6]:
# list of top level codes (e.g., '001-139', ...)
toplevelnodes = tree.children
toplevelcodes = [node.code for node in toplevelnodes]
print '\t'.join(toplevelcodes)


001-139	140-239	240-279	290-319	320-389	390-459	460-519	520-579	580-629	630-679	680-709	710-739	760-779	780-789	790-796	797	798	799	800-999	V01-V06	V07-V09	V10-V19	V20-V29	V30-V39	V40-V49	V50-V59	V60-V69	V70-V82	V83-V84	V85	V86	V87	V88	V89	E979	E849	E800-E807	E810-E819	E820-E825	E826-E829	E830-E838	E840-E845	E846-E848	E850-E858	E860-E869	E870-E876	E878-E879	E880-E888	E890-E899	E900-E909	E910-E915	E916-E928	E929	E930-E949	E959	E956	E954	E950	E951	E952	E953	E955	E957	E958	E960-E969	E970-E978	E980-E989	E990-E999


In [17]:
# get english description of ICD9 code
# prints: 'Cholera due to vibrio cholerae el tor'
#tree.find('001.1').description

# prints: 'ROOT'
#tree.description

# prints: 'Cholera'
#tree.find('001.1').parent.description

# also prints: 'Cholera'
tree.find('001').description


u'Cholera'

### get most common diagnoses in MIMIC-III dataset:

In [29]:
#first split up the ICD_code column on "-":
#https://gist.github.com/jlln/338b4b0b55bd6984f883

def splitDataFrameList(df,target_column,separator):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df


In [30]:
mimic_parsed = splitDataFrameList(mimic,'ICD9_CODE','-')


,Unnamed: 0,HADM_ID,SUBJECT_ID,ICD9_CODE,CHARTDATE,DESCRIPTION,TEXT,Level2ICD,TopLevelICD,V9
0,1,167853,22532,01193-4254-42731-2639-2762-5070-5119-2113,2151-08-04,Report,Admission Date: [**2151-7-16**] Dischar...,NaN,11-425-427-263-276-507-511-211,cat:1-cat:8-cat:3-cat:9-cat:2
1,2,107527,13702,5191-49121-51881-486-2761-2449-311,2118-06-14,Report,Admission Date: [**2118-6-2**] Discharg...,NaN,519-491-518-486-276-244-311,cat:9-cat:3-cat:5
2,3,167118,13702,5191-5185-496-2762-45340-5533,2119-05-25,Report,Admission Date: [**2119-5-4**] D...,NaN,519-518-496-276-453-553,cat:9-cat:3-cat:8-cat:10
3,4,196489,13702,51884-5849-34830-49121-2760-4160-3594-5780-276...,2124-08-18,Report,Admission Date: [**2124-7-21**] ...,NaN,518-584-348-491-276-416-359-578-530-487-553-V0...,cat:9-cat:11-cat:6-cat:3-cat:8-cat:10-cat:19
4,5,135453,26880,80506-5070-42823-2930-4538-E882-4280-4011-V450...,2162-03-25,Report,Admission Date: [**2162-3-3**] D...,NaN,805-507-428-293-453-E882-401-V4502-427-V1254-8...,cat:18-cat:9-cat:8-cat:5-cat:19-cat:17


In [35]:
mimic_parsed.shape

(618998, 10)

In [33]:
list(mimic_parsed)

['CHARTDATE',
 'DESCRIPTION',
 'HADM_ID',
 'ICD9_CODE',
 'Level2ICD',
 'SUBJECT_ID',
 'TEXT',
 'TopLevelICD',
 'Unnamed: 0',
 'V9']

In [40]:
mimic2 = mimic_parsed.groupby('ICD9_CODE').count()
mimic2

,CHARTDATE,DESCRIPTION,HADM_ID,Level2ICD,SUBJECT_ID,TEXT,TopLevelICD,Unnamed: 0,V9
ICD9_CODE,,,,,,,,,
0030,2,2,2,0,2,2,2,2,2
0031,1,1,1,0,1,1,1,1,1
0038,2,2,2,0,2,2,2,2,2
0039,1,1,1,0,1,1,1,1,1
0041,3,3,3,0,3,3,3,3,3
0048,1,1,1,0,1,1,1,1,1
0049,1,1,1,0,1,1,1,1,1
0051,1,1,1,0,1,1,1,1,1
00581,2,2,2,0,2,2,2,2,2


In [42]:
mimic2.reset_index(inplace=True)
mimic2

,ICD9_CODE,CHARTDATE,DESCRIPTION,HADM_ID,Level2ICD,SUBJECT_ID,TEXT,TopLevelICD,Unnamed: 0,V9
0,0030,2,2,2,0,2,2,2,2,2
1,0031,1,1,1,0,1,1,1,1,1
2,0038,2,2,2,0,2,2,2,2,2
3,0039,1,1,1,0,1,1,1,1,1
4,0041,3,3,3,0,3,3,3,3,3
5,0048,1,1,1,0,1,1,1,1,1
6,0049,1,1,1,0,1,1,1,1,1
7,0051,1,1,1,0,1,1,1,1,1
8,00581,2,2,2,0,2,2,2,2,2
9,0059,1,1,1,0,1,1,1,1,1


In [43]:
list(mimic2)

['ICD9_CODE',
 'CHARTDATE',
 'DESCRIPTION',
 'HADM_ID',
 'Level2ICD',
 'SUBJECT_ID',
 'TEXT',
 'TopLevelICD',
 'Unnamed: 0',
 'V9']

In [44]:
mimic2.drop(['DESCRIPTION',
 'HADM_ID',
 'Level2ICD',
 'SUBJECT_ID',
 'TEXT',
 'TopLevelICD',
 'Unnamed: 0',
 'V9'], axis=1, inplace=True)
mimic2

,ICD9_CODE,CHARTDATE
0,0030,2
1,0031,1
2,0038,2
3,0039,1
4,0041,3
5,0048,1
6,0049,1
7,0051,1
8,00581,2
9,0059,1


In [45]:
mimic2.rename(columns={'CHARTDATE':'count'}, inplace=True)
mimic2

,ICD9_CODE,count
0,0030,2
1,0031,1
2,0038,2
3,0039,1
4,0041,3
5,0048,1
6,0049,1
7,0051,1
8,00581,2
9,0059,1


In [46]:
mimic2.sort_values('count', inplace=True, ascending=False)
display(mimic2[:5])

,ICD9_CODE,count
1957,4019,20053
2104,4280,12842
2093,42731,12594
2014,41401,12179
2950,5849,8907


In [68]:
mimic2.rename(columns={'conceptID':'conceptId'}, inplace=True)
mimic2[:2]

,conceptId,count
1957,4019,20053
2104,4280,12842


In [71]:
code_desc = pd.merge(mimic2,desc[['conceptId','term']]
                                   ,on='conceptId', how='left')
code_desc[:10]

,conceptId,count,term
0,4019,20053,Hypertension NOS
1,4280,12842,CHF NOS
2,42731,12594,Atrial fibrillation
3,41401,12179,Crnry athrscl natve vssl
4,5849,8907,Acute kidney failure NOS
5,25000,8784,DMII wo cmp nt st uncntr
6,2724,8504,Hyperlipidemia NEC/NOS
7,51881,7249,Acute respiratry failure
8,5990,6442,Urin tract infection NOS
9,53081,6156,Esophageal reflux


In [72]:
code_desc.to_csv('mimic_code_counts', sep='\t', index=False)